This notebook will be used for the capstone project.

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Segmenting and Clustering Neighbourhoods in Toronto

In [2]:
# returns a list of dataframes
data_orig = pd.read_html("http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")#, attrs = {'id': 'table'})

# get the table
data_orig = data_orig[0]
data_orig.head(12)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [3]:
# make a copy and work on this from now on
data = data_orig.copy(deep=True)

In [4]:
data.shape

(287, 3)

In [5]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
data.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [6]:
# Ignore cells with a borough that is "Not assigned"
data = data[data['Borough']!='Not assigned']

# If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough
neighborhoods_to_replace = data['Neighborhood']=='Not assigned'
data.loc[neighborhoods_to_replace, 'Neighborhood'] = data.loc[neighborhoods_to_replace, 'Borough']


In [7]:
# More than one neighborhood can exist in one postal code area. These neighborhoods
# should be combined into one row and separated with a comma.

def aggregate_neighbourhoods(df):
    # create a dictionary to hold the postal codes as keys and (Boroughs, Neighborhoods) as values
    D = dict.fromkeys(data['PostalCode'].unique())
    # loop through the rows of the dataframe df
    for item in df:
        # get the list of Neighborhoods and join the list items with ", "
        hds = ", ".join(list(item[1]['Neighborhood'].unique()))
        # now get the list of boroughs and join the list items with ", "
        bor = ", ".join(list(item[1]['Borough'].unique()))
        # populate the dictionary
        D[item[0]] = (bor, hds)
        
    new_df = pd.DataFrame.from_dict(D, orient='index')  # without orient='index' the dict keys will be the columns of new_df
    new_df.reset_index(inplace=True)  # reset the index to row numbers instead of postal codes
    new_df.columns = ['Postal Code', 'Borough', 'Neighborhood']  # name the columns
    return new_df


df = data.groupby(['PostalCode'])  # group the data by the postal codes
data = aggregate_neighbourhoods(df)  # pass the dataframe in the function to get the final dataframe
data.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [8]:
data.shape

(103, 3)

In [10]:
geo_coords = pd.read_csv('../data/Geospatial_Coordinates.csv')

# sanity check. The below should not return an error if the dataframe data is correct
assert(geo_coords.shape[0] == data.shape[0])

geo_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476



Merge the geo_coords and data on the 'Postal Code' which is common in both dataframes:

In [11]:
analysis_data = data.merge(geo_coords, on='Postal Code')
analysis_data.head(12)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


**Guidelines:**
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

* to add enough Markdown cells to explain what you decided to do and to report any observations you make.
* to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [12]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [13]:
# set number of clusters
kclusters = 7

analysis_data_clustering = analysis_data.drop(['Postal Code', 'Borough', 'Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(analysis_data_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 5, 0, 1, 2, 3, 4, 0], dtype=int32)

In [14]:
analysis_data.insert(0, 'Cluster Labels', kmeans.labels_)
analysis_data.head()

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,0,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [15]:
# create map
toronto_latitude = 43.6532
toronto_longitude = -79.3832
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(analysis_data['Latitude'], analysis_data['Longitude'], analysis_data['Neighborhood'], analysis_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters